## Imports

In [1]:
# pip install --upgrade scikit-learn

In [103]:
import time
from IPython.display import clear_output
import numpy    as np
import pandas   as pd
import seaborn  as sb
import matplotlib.pyplot as plt
import sklearn  as skl

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import model_selection # train_test_split
from sklearn import metrics         # accuracy_score, balanced_accuracy_score, plot_confusion_matrix
from sklearn import set_config

set_config(display='diagram') # Useful for display the pipeline

print("Pandas  ", pd.__version__)
print("Sklearn ", skl.__version__) # Try to use 0.24

Pandas   1.2.0
Sklearn  0.24.1


## Get the dataset
- **CLOUD = True**: Download dataset from Kaggle. Necesary for cloud enviroments like COLAB. **Specify your [kaggle credentials](https://www.kaggle.com/docs/api)**.
- **CLOUD = False**: Get the dataset from your local machine. **Specify the data path**.

In [104]:
CLOUD = False

if CLOUD:
    import os
    os.environ['KAGGLE_USERNAME'] = "saimohandalli "
    os.environ['KAGGLE_KEY']      = "a7b83b8b59e74017d964a2d5ed281128"  # See https://www.kaggle.com/docs/api
    !pip install --upgrade kaggle
    !kaggle competitions download -c titanic
    DATA_PATH = "./"

else:
    DATA_PATH = "../../datasets/titanic/"

## Load data

In [105]:
df      = pd.read_csv(DATA_PATH + "train.csv", index_col='PassengerId')
df_test = pd.read_csv(DATA_PATH + "test.csv",  index_col='PassengerId')

print("Train DataFrame:", df.shape)
print("Test DataFrame: ", df_test.shape)

Train DataFrame: (891, 11)
Test DataFrame:  (418, 10)


## Check missings

In [106]:
df.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [107]:
df_test.isnull().sum()

Pclass        0
Name          0
Sex           0
Age          86
SibSp         0
Parch         0
Ticket        0
Fare          1
Cabin       327
Embarked      0
dtype: int64

# Exercise 1:
Extract the title (Mr, Mrs, ... ) from the "Name" column.

Tips:
- split(',')[1] to get the 2nd part, and remove the surnamename
- split('.')[0] to get the 1str part, and remove the name

In [7]:
# USING FOR LOOP

list = df.Name.str.split(',')
list1 = []
title = []
for i in list:
    list1.append(i[1].split('.'))
for j in list1:
    title.append(j[0])
print(type(np.array(title)))
    


<class 'numpy.ndarray'>


In [8]:
list = df_test.Name.str.split(',')
list1 = []
title1 = []
for i in list:
    list1.append(i[1].split('.'))
for j in list1:
    title1.append(j[0])

In [112]:
df['Title'] = np.array(title)
df_test["Title"] = np.array(title1)

In [113]:
# CODE HERE get_Title_from_Name funtion
df["Title"]      = df['Name'].map(lambda x: x.split(',')[1].split('.')[0])
df_test["Title"] = df_test['Name'].map(lambda x: x.split(',')[1].split('.')[0])


In [115]:
print(df.Title.head())
print('+++++++++++++++++++++++++++++++')
print(df_test.Title.head())

PassengerId
1       Mr
2      Mrs
3     Miss
4      Mrs
5       Mr
Name: Title, dtype: object
+++++++++++++++++++++++++++++++
PassengerId
892      Mr
893     Mrs
894      Mr
895      Mr
896     Mrs
Name: Title, dtype: object


# Exercise 2:
Apply the title_dictionary to get a better information about the title. You have to overwrite the Title variable.

In [116]:
df.Title.unique()

array([' Mr', ' Mrs', ' Miss', ' Master', ' Don', ' Rev', ' Dr', ' Mme',
       ' Ms', ' Major', ' Lady', ' Sir', ' Mlle', ' Col', ' Capt',
       ' the Countess', ' Jonkheer'], dtype=object)

In [117]:
title_dictionary = {
    " Capt": "Officer",
    " Col": "Officer",
    " Major": "Officer",
    " Jonkheer": "Royalty",
    " Don": "Royalty",
    " Sir" : "Royalty",
    " Dr": "Officer",
    " Rev": "Officer",
    " the Countess":"Royalty",
    " Mme": "Mrs",
    " Mlle": "Miss",
    " Ms": "Mrs",
    " Mr" : "Mr",
    " Mrs" : "Mrs",
    " Miss" : "Miss",
    " Master" : "Master",
    " Lady" : "Royalty"
}

In [118]:
# CODE HERE
df['Title'].replace(title_dictionary,inplace = True)
df_test['Title'].replace(title_dictionary,inplace = True)


In [119]:
print(df.Title.unique())
print('+++++++++++++++++++++++++++++++')
df_test.Title.unique()

['Mr' 'Mrs' 'Miss' 'Master' 'Royalty' 'Officer']
+++++++++++++++++++++++++++++++


array(['Mr', 'Mrs', 'Miss', 'Master', 'Officer', ' Dona'], dtype=object)

# Exercise BONUS:
Try to extract some information from the feature **Ticket**

# Exercise BONUS:
Try to extract some information from the feature **Cabin**

# Preprocessing
For X data:
- We drop Survived because is the target variable
- We drop Name because we have extracted the Title: Mr, Mrs, ...
- We drop Ticket because it has no information -> see df.Ticket.nunique()
- We drop Cabin because it has a lot of missings (77% are missings)

Then, we identify **numerical** variables and **categorical** variables,

In [126]:
x = df.drop(columns=["Survived", 'Name', 'Ticket', 'Cabin']) # X DATA (WILL BE TRAIN+VALID DATA)
y = df["Survived"] # 0 = No, 1 = Yes

x_test = df_test.drop(columns=['Name', 'Ticket', 'Cabin']) # # X_TEST DATA (NEW DATA)

In [127]:
cat_vars  = ['Sex', 'Embarked', 'Title']         # x.select_dtypes(include=[object]).columns.values.tolist()
num_vars  = ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age'] # x.select_dtypes(exclude=[object]).columns.values.tolist()

print("\nNumerical features:\n", num_vars)
print("\nCategorical features:\n", cat_vars)


Numerical features:
 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age']

Categorical features:
 ['Sex', 'Embarked', 'Title']


# Exercise 3:
Create a **ColumnTransformer for Tree Models**. Remember:
- Categorical: Some SimpleImputer -> Some Encoder
- Numerical: Some SimpleImputer -> NO Encoder

In [128]:
num_preprocessing = pipeline.Pipeline(steps=[
    ('imputer', impute.SimpleImputer(strategy = 'mean'))
])

cat_preporcessing = pipeline.Pipeline(steps=[
    ('imputer',impute.SimpleImputer(strategy = 'constant',fill_value = 'missing')),
    ('onehot',preprocessing.OneHotEncoder(handle_unknown = 'ignore'))
])

tree_prepro = compose.ColumnTransformer(transformers=[
    ('num', num_preprocessing, num_vars),
    ('cat', cat_preporcessing, cat_vars),
], remainder='drop') # Drop other vars not specified in num_vars or cat_vars

tree_prepro

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer())]),
                                 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['Sex', 'Embarked', 'Title'])])

# Exercise 4
1. Complete the diccionary with some Tree Models.
2. Then we put each model in a Pipeline where:
   - first is the prepocessing with the column Transformer
   - Then is the Tree model
3. Display the fullpipeline of the LGBMClassifier

In [134]:
from sklearn.tree          import DecisionTreeClassifier
from sklearn.ensemble      import RandomForestClassifier
from sklearn.ensemble      import ExtraTreesClassifier
from sklearn.ensemble      import AdaBoostClassifier
from sklearn.ensemble      import GradientBoostingClassifier
from sklearn.experimental  import enable_hist_gradient_boosting # Necesary for HistGradientBoostingClassifier
from sklearn.ensemble      import HistGradientBoostingClassifier
from xgboost               import XGBClassifier
from lightgbm              import LGBMClassifier
from catboost              import CatBoostClassifier

In [136]:
tree_classifiers = {
  "Decision Tree": DecisionTreeClassifier(),
  "Extra Trees":   ExtraTreesClassifier(),
  "Random Forest": RandomForestClassifier(),
  "AdaBoost":      AdaBoostClassifier(),
  "Skl GBM":       GradientBoostingClassifier(),
  "Skl HistGBM":   HistGradientBoostingClassifier(),
  "LightGBM":      LGBMClassifier(),
  #"CatBoost":      CatBoostClassifier(),
   # "XGBoost":      XGBClassifier(verbose = 1)
}
tree_classifiers = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in tree_classifiers.items()}
tree_classifiers['LightGBM']


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['Pclass', 'SibSp', 'Parch',
                                                   'Fare', 'Age']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Sex', 'Embarked',
                                                   'Title'])])),
                ('lgbmclassifier', LGBMClassifier())])

# Exercise 5:
Define a simple split validation strategy with:
- 80% for train
- 20% for validation
- With stratification
- random_state=0

And train all the models in a for loop

In [168]:
x_train, x_val, y_train, y_val = model_selection.train_test_split(
    x,y,
    test_size = 0.2,
    stratify = y,
    random_state = 0
)

results = pd.DataFrame({'model_name':[],'Accuracy':[],'Balanced Accuracy':[],'time':[]})
for model_name, model in tree_classifiers.items():
    # CODE HERE
    # TRAIN PIPELINE (PREPRO + MODEL) WITH TRAIN DATA
    # EVAL PIPELINE WITH VAL DATA (SEE ACCURACY AND BALANCED_ACCURACY)
    start_time = time.time()
    model.fit(x_train,y_train)
    y_pred = model.predict(x_val)
    end_time = time.time() - start_time
    results = results.append({'model_name':model_name,
             'Accuracy':metrics.accuracy_score(y_val,y_pred),
             'Balanced Accuracy': metrics.balanced_accuracy_score(y_val,y_pred),
             'time':end_time},
              ignore_index = True)
    
    results_ord = results.sort_values(by=['Accuracy'],ascending = False,ignore_index = True)
    results_ord.index +=1
    clear_output(wait = True)
    display(results_ord.style.bar(subset=['Accuracy','Balanced Accuracy'],vmin=0,vmax = 100,color = '#5fba7d'))
#print(results_ord)
    

,model_name,Accuracy,Balanced Accuracy,time
1,Skl GBM,0.837989,0.811462,0.103775
2,LightGBM,0.810056,0.783333,0.068816
3,Skl HistGBM,0.804469,0.778788,0.414890
4,AdaBoost,0.798883,0.774242,0.093749
5,Decision Tree,0.793296,0.769697,0.015981
6,Random Forest,0.782123,0.755204,0.152625
7,Extra Trees,0.748603,0.725231,0.136634


# Exercise 6:
Define a 10 Fold cross validation strategy with:
- With stratification
- shuffle=True
- random_state=0

And train all the models in a for loop.

Tip you can use **[cross_val_predict](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html)** for both training and predict with 

In [170]:
skf = model_selection.StratifiedKFold(
    n_splits=10,
    shuffle=True,
    random_state=0
)

results = pd.DataFrame({'model_name':[],'Accuracy':[],'Balanced Accuracy':[],'time':[]})
for model_name, model in tree_classifiers.items():
    # CODE HERE
    # TRAIN PIPELINE (PREPRO + MODEL) WITH TRAIN DATA
    # EVAL PIPELINE WITH VAL DATA (SEE ACCURACY AND BALANCED_ACCURACY)
    start_time = time.time()
    y_pred = model_selection.cross_val_predict(model,x,y,cv=skf)
    end_time = time.time() - start_time
    results = results.append({'model_name':model_name,
             'Accuracy':metrics.accuracy_score(y,y_pred),
             'Balanced Accuracy': metrics.balanced_accuracy_score(y,y_pred),
             'time':end_time},
              ignore_index = True)
    
    results_ord = results.sort_values(by=['Accuracy'],ascending = False,ignore_index = True)
    results_ord.index +=1
    clear_output(wait = True)
    display(results_ord.style.bar(subset=['Accuracy','Balanced Accuracy'],vmin=0,vmax = 100,color = '#5fba7d'))

,model_name,Accuracy,Balanced Accuracy,time
1,Skl GBM,0.838384,0.817036,1.076111
2,LightGBM,0.827160,0.811236,0.605381
3,Skl HistGBM,0.823793,0.804645,4.637677
4,Random Forest,0.813692,0.798102,1.489050
5,AdaBoost,0.810325,0.799228,0.828754
6,Extra Trees,0.801347,0.786981,1.417241
7,Decision Tree,0.789001,0.778617,0.167578


# Exercise 7
Train **with all data** the best model

In [173]:
best_model = tree_classifiers['Skl GBM']
best_model.fit(x,y)
# Select your best model
# Train with all data your best model

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['Pclass', 'SibSp', 'Parch',
                                                   'Fare', 'Age']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Sex', 'Embarked',
                                                   'Title'])])),
                ('gradientboostingclassifier', GradientBoostingClassifier())])

# Exercise 8
With your best model, generate the predicitions for test data (x_test)

In [174]:
test_pred = best_model.predict(x_test)
# Get the predictions for x_test

# Exercise 9

Submit to kaggle using the kaggle API. And send us your score. You can try to improve it.

In [175]:
sub = pd.DataFrame(test_pred, index=x_test.index, columns=["Survived"])
sub.head()

,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,1


In [176]:
sub.to_csv("sub.csv")

In [179]:
!kaggle competitions submit -c titanic -f sub.csv -m "My submission message"

'kaggle' is not recognized as an internal or external command,
operable program or batch file.


# Exercise BONUS

Knowing how to export your models is very important for putting models in production. Try to
- Export and Load the ColumTransformer in pickle
- Export and Load the ColumTransformer in joblib
- Export and load the Pipeline